In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import matplotlib.patches as patches
# sys.path.append("..")

from Models.splinedist import *
import pytorch_lightning as pl
import torch
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor, RichProgressBar, RichModelSummary
from Datasets.CISD import * 
from ray_lightning import RayPlugin
from ray_lightning import RayShardedPlugin
from ray_lightning.tune import TuneReportCallback, get_tune_resources
from ray import tune

In [2]:
path = path=os.path.join(os.getcwd(),'Datasets/CISD/CISD.json')
root = os.path.join(os.getcwd(), "Datasets/CISD/center_slice")
savepath = os.path.join(os.getcwd(), "Models_final2/weights2")
logdir = os.path.join(os.getcwd(), "pl_logs_final2")
path, root, savepath, logdir

('/master/home/mbenimam/SplineDist/src/Datasets/CISD/CISD.json',
 '/master/home/mbenimam/SplineDist/src/Datasets/CISD/center_slice',
 '/master/home/mbenimam/SplineDist/src/Models_final2/weights2',
 '/master/home/mbenimam/SplineDist/src/pl_logs_final2')

In [4]:


# def objective(config):
    
from concurrent.futures import ThreadPoolExecutor




# Create your PTL model.

def process(trainer, model, data):
    return trainer.fit(model, data)


    
    
data = CISD_datamodule(path,
                       root=root,
                       contourSize=300)

config = {
    "lambda1": tune.loguniform(1e-4, 1),
    "lambda2": tune.loguniform(1e-6, 1e-1),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "num_control_points": tune.choice([8, 16, 32, 64]),
    "object_threshold":0.4,
    "nms_threshold": 0.1,
    "kernel_size": tune.choice([1, 3, 5]),
    "contourSize": 300,
    "weight_pos": tune.loguniform(0.1, 1),
    "num_features": tune.choice([64, 128, 256]),
}

models = [None]*6
trainers = [None]*6

for i, gpu in enumerate([1, 3, 4, 5, 6, 7]):
    
    logger = pl_loggers.TensorBoardLogger(
        save_dir=logdir,
        version=i,
        name="spline_dist/CISD-trial",
        log_graph=True,
        default_hp_metric=True)
    
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        filename="splineDist-CISD{config}-{epoch:02d}-{loss_val:.2f}",
        monitor="loss_val",
        save_top_k=2,
        mode="min",
        dirpath=savepath)
    callbacks=[checkpoint_callback]
    
#     models[i] = SplineDist(lambda1=config["lambda1"],
#                 lambda2=config["lambda2"],
#                 learning_rate=config["learning_rate"],
#                 num_control_points=config["num_control_points"],
#                 object_threshold=config["object_threshold"],
#                 kernel_size=config["kernel_size"],
#                 contourSize=config["contourSize"],
#                 nms_threshold=config["nms_threshold"],
#                 weight_pos=config["weight_pos"],
#                 num_features=config["num_features"])
    
    models[i] = SplineDist(lambda1=0.9,
            lambda2=1e-4,
            learning_rate=1e-4,
            num_control_points=32,
            object_threshold=0.3,
            kernel_size=7,
            contourSize=300,
            nms_threshold=0.1,
            weight_pos=1,
            num_features=128)
    
    trainers[i] = pl.Trainer(
        gpus=[gpu],
        max_epochs=150,
        logger = logger,
        callbacks=callbacks,
        num_sanity_val_steps=0)
    
    

with ThreadPoolExecutor(max_workers=16) as executor:
    futures = []
    for i, gpu in enumerate([1, 3, 4, 5, 6, 7]):
        futures.append(executor.submit(process, trainers[i], models[i], data))

    for future in futures:
        future.result() # this will block


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 

(device(type='cuda', index=4),) {}
(device(type='cuda', index=1),) {}
(device(type='cuda', index=5),) {}
(device(type='cuda', index=3),) {}
(device(type='cuda', index=6),) {}
(device(type='cuda', index=7),) {}


/master/home/mbenimam/.local/lib/python3.8/site-packages/torch/_jit_internal.py:668: LightningDeprecationWarning: The `LightningModule.loaded_optimizer_states_dict` property is deprecated in v1.4 and will be removed in v1.6.
  if hasattr(mod, name):
/master/home/mbenimam/.local/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/master/home/mbenimam/SplineDist/src/Models/unet.py:62: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
/master/home/mbenimam/SplineDist/s

Training: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
# torch.cuda.empty_cache()

config = {
    "lambda1": tune.loguniform(1e-4, 1),
    "lambda2": tune.loguniform(1e-6, 1e-1),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "num_control_points": tune.choice([8, 16, 32, 64]),
    "object_threshold":0.4,
    "nms_threshold": 0.1,
    "kernel_size": tune.choice([1, 3, 5]),
    "contourSize": 300,
    "weight_pos": tune.loguniform(0.1, 1),
    "num_features": tune.choice([64, 128, 256]),
}


analysis = tune.run(
        objective,
        metric="loss",
        mode="min",
        config=config,
        num_samples=10,
        resources_per_trial={'gpu': 1, "cpu": 2},
        name="Tune_CISD")



In [ ]:
tune.loguniform(1e-4, 1)